In [ ]:
import os
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("./spam.csv", encoding='latin-1')
clean = df.drop(['Unnamed: 2'], axis=1)
clean = clean.drop(['Unnamed: 3'], axis = 1)
clean = clean.drop(['Unnamed: 4'], axis = 1)
clean.rename(columns = {'v1' : 'Category', 'v2' : 'Message'}, inplace=True)
clean["Message"] = clean["Message"].str.replace('[^a-zA-Z]','', regex = True)
clean["Message"] = clean["Message"].str.lower()

In [ ]:

clean